In [10]:
!pip install labelbox[data]
!pip install jsonlines

In [7]:
import labelbox as lb
import labelbox.types as lb_types
import uuid
import base64
import requests

# Setup client
with open("labelbox_api_key.txt","r") as f:
    API_KEY = f.read().strip()
client = lb.Client(api_key=API_KEY)

# Get ontology
print("===ONTOLOGY DETAILS===")
ontology = client.get_ontology("clqo6bd8v0jc407ybc1r9ehlb")
print("Name: ", ontology.name)
tools = ontology.tools()

# for tool in tools:
#   print(tool)

# Get project
print("\n===PROJECT DETAILS===")
PROJECT_ID = 'clqoh3ylw1o8s070hd6ch5z7o' # WHOI RSI USVI Fish
# PROJECT_ID = 'clqo7auln0mpo07wphorp0t2e' # Test WHOI RSI USVI Fish
project = client.get_project(PROJECT_ID)
print("Name: ", project.name)

# Get dataset
DATASET_ID = "clqh7v7qi001r07886j6aws7i"
dataset = client.get_dataset(DATASET_ID)
print("\n===DATASET DETAILS===")
print("Name: ", dataset.name)

# Dataset parameters
species_level = True # Extract species-level data (otherwise single-class "fish"), use with data_rows_done_only
data_rows_done_only = True # Only utilize data rows that have undergone species review

===ONTOLOGY DETAILS===
Name:  WHOI-RSI-USVI-Fish

===PROJECT DETAILS===
Name:  WHOI-RSI-USVI-Fish-detect-and-track

===DATASET DETAILS===
Name:  imerit-26102023-3fps-clips


In [18]:
# Enumerate species labels for YOLO class formatting
classes_option = {}
classes_enum = {}
ordered_class_names = []

classes_option["fish"] = {"label": "Fish", "value": "fish"}
classes_enum["fish"] = 0
ordered_class_names.append("fish")

if species_level:
    for option_num, option in enumerate(tools[0].classifications[0].options):
        classes_option[option.value] = option
        classes_enum[option.value] = len(ordered_class_names)
        ordered_class_names.append(option.value)
print("Classes: ", len(ordered_class_names))
print(classes_option)

Classes:  104
{'fish': {'label': 'Fish', 'value': 'fish'}, 'absa_sergeant_major': Option(value='absa_sergeant_major', label='ABSA (Sergeant Major)', schema_id='clr6rzlg40jca07yvbsh21gix', feature_schema_id='clr6rzlg40jc907yvaa6c74uy', options=[]), 'acan_unknown': Option(value='acan_unknown', label='ACAN_UNKNOWN (Unknown Tang (Acanthuridae))', schema_id='clr6rzlg40jcc07yvdvimbb07', feature_schema_id='clr6rzlg40jcb07yv2wiu7we2', options=[]), 'acch_doctorfish': Option(value='acch_doctorfish', label='ACCH (Doctorfish)', schema_id='clr6rzlg40jce07yvc1bf39h4', feature_schema_id='clr6rzlg40jcd07yv93yo4448', options=[]), 'acco_blue_tang': Option(value='acco_blue_tang', label='ACCO (Blue Tang)', schema_id='clr6rzlg40jcg07yvhpdl0y1h', feature_schema_id='clr6rzlg40jcf07yvek4v76ng', options=[]), 'acpo_honeycomb_cowfish': Option(value='acpo_honeycomb_cowfish', label='ACPO (Honeycomb Cowfish)', schema_id='clr6rzlg40jci07yv74smh79v', feature_schema_id='clr6rzlg40jch07yv8mhz9rsb', options=[]), 'acqu_s

In [15]:
## Recommended to download JSON from Labelbox using the Browser Interface
# TODO: Verify whether or not this includes interpolated (non-keyframed) data

# Extracts fish labels from labelbox json file and converts them into YOLO format
# Assumes the global_key from labelbox matches the directory structure of the images
# Fish class is assumed as 0

dry_run = False
save_images = False
make_copy = True # Good for creating sub-datasets (like species-classifier, since not all videos have been labelled to that level)

import json
import jsonlines
import os
from pathlib import Path
from bbox_utils import *
from tqdm import tqdm
import shutil

json_path = "/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/Labelbox-Export-WHOI-RSI-USVI-Fish-detect-and-track - 6_11_2024.ndjson"
image_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-yolo-dataset/images")
# label_output_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-yolo-dataset/labels")
# label_output_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-yolo-dataset/labels_species_only")
# label_output_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-yolo-dataset/test_labels")
# image_output_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-yolo-dataset/test_images")

# For species-level datasets
label_output_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-species-yolo-dataset/labels")
image_output_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-species-yolo-dataset/images")

list_of_videos = []
stats = {}

# with open(json_path, "r") as f:
with jsonlines.open(json_path, "r") as json_file:

    # Iterate through each video in the JSON file
    for i, datarow in tqdm(enumerate(json_file)):
        global_key = datarow["data_row"]["global_key"]
        
        # Skip datarows that are not DONE, if applicable (usually used alongside species-level)
        project_status = datarow["projects"][PROJECT_ID]["project_details"]["workflow_status"]
        if data_rows_done_only and not project_status == "DONE":
            print("Skipping not done: ", global_key)
            continue

        img_sz = (datarow["media_attributes"]["width"], datarow["media_attributes"]["height"])
        
        # Video path
        vid_path = Path(global_key)
        rel_vid_path = vid_path.parent / "_".join(vid_path.stem.split("_")[:-1])

        # Grab frame labels
        try:
            frames_json = datarow["projects"][PROJECT_ID]["labels"][0]["annotations"]["frames"]
        except:
            # No labels in this video, so continue
            print("Skipping ", global_key, " has no labels")
            continue

        # Iterate through frames
        frame_count = datarow["media_attributes"]["frame_count"]
        
        for frame_id in range(frame_count):
            img_name = "frame_%03d"%(int(frame_id))
            img_path = image_root / rel_vid_path / (img_name + ".png")
            output_path = label_output_root / rel_vid_path / (img_name + ".txt")  

            if make_copy:
                output_img_path = image_output_root / rel_vid_path / (img_name + ".png")

                if not dry_run:
                    os.makedirs(output_img_path.parent, exist_ok=True)
                    shutil.copy2(img_path, output_img_path)
                else:
                    print("Copying ", img_path, " to ", output_img_path)
                
            # Verify this image exists
            assert img_path.exists(), f"Image not found {img_path}"
            
            os.makedirs(output_path.parent, exist_ok=True)
            
            # Make label file, overwrite if already there
            open(output_path, "w")

            # No labels in this frame, so continue
            lblbox_frame_id = frame_id + 1
            if str(lblbox_frame_id ) in frames_json:
                # There is an off-by-one naming scheme here
                frame_data = frames_json[str(lblbox_frame_id)]
            else:
                continue

            for object_id, object_data in frame_data["objects"].items():
                lbl_bbox = object_data["bounding_box"]
                
                if species_level:
                    if len(object_data["classifications"]) > 0:
                        class_name = object_data["classifications"][0]["radio_answer"]["value"]
                        label = classes_enum[class_name]
                    else:
                        label = 0
                else:
                    label = 0

                yolo_bbox = list(labelbox2yolo_bbox(lbl_bbox, img_sz))
                yolo_bbox.insert(0, label) # Fish class for now
                with open(output_path, "a") as f:
                    if not dry_run:
                        f.write(" ".join(map(str, yolo_bbox)))
                        f.write("\n")

        list_of_videos.append(global_key)
print("done")

118it [13:49,  7.03s/it]

done


In [18]:
print(len(list_of_videos))

64


In [20]:
# Create yolov5 dataset configuration yaml
import yaml

project_root = Path("/srv/warplab/shared/datasets/WHOI_RS_Fish_Detector/whoi-rsi-fish-detection-yolo-dataset/")
train_split_filename = "train_species_split.txt"
val_split_filename = "val_species_split.txt"
test_split_filename = "test_species_split.txt"
dataset_yaml_filename = "fish_species_yolo_dataset.yaml"

if species_level:
    names = dict(zip(classes_enum.values(), classes_enum.keys()))
else:
    names = {0: "fish"}

yolo_dataset = {
    "path": str(project_root),
    "train": f"./{train_split_filename}",
    "val": f"./{val_split_filename}",
    "test": f"./{test_split_filename}",
    "names": names,
}
yaml_data = yaml.dump(yolo_dataset)
with open(project_root / dataset_yaml_filename, "w") as f:
    f.write(yaml_data)

In [33]:
# Create train, val, and test splits
open(project_root / train_split_filename, "w")
open(project_root / val_split_filename, "w")
open(project_root / test_split_filename, "w")

import glob

# Test split contains only years 2016 and 2017, these dates are inferred from the global_key
img_paths = glob.iglob(str(project_root / "**/*.png"), recursive=True)

# Note: this only adds images that have corresponding labels
split_stats = {}
split_stats["num_train_frames"] = 0
split_stats["num_val_frames"] = 0
split_stats["num_test_frames"] = 0


for img_path in tqdm(img_paths):
    # Get relative path starting at project root
    project_root_parts = len(project_root.parts)

    frame_path = Path(*Path(img_path).parts[project_root_parts+1:])
    project_img_path = "images" / frame_path
    
    vid_path = project_img_path.parent
    
    if not (project_root / "labels" / frame_path.with_suffix(".txt")).exists():
        continue
    
    with open(project_root / "video_list.txt", "a") as f:
        f.write(str(vid_path) + "\n")
        
    # Test split
    if "2016" in str(project_img_path) or "2017" in str(project_img_path):
        with open(project_root / test_split_filename, "a") as f:
            f.write("./" + str(project_img_path) + "\n")
        split_stats["num_test_frames"] += 1
            
    # Val split
    elif "2018" in str(project_img_path):
        with open(project_root / val_split_filename, "a") as f:
            f.write("./" + str(project_img_path) + "\n")
        split_stats["num_val_frames"] += 1
    
    # Train split
    else:
        with open(project_root / train_split_filename, "a") as f:
            f.write("./" + str(project_img_path) + "\n")
        split_stats["num_train_frames"] += 1
        
print("done")
print(split_stats)


14670it [04:13, 57.77it/s] 

done
{'num_train_frames': 1890, 'num_val_frames': 630, 'num_test_frames': 3240}
